# Content-based Filtering

In [ ]:
import pandas as pd
import numpy as np
import ast
import time

users = pd.read_csv('game_data_prepared/users.csv')
ratings = pd.read_csv('game_data_prepared/ratings.csv')
gamesF = pd.read_csv('game_data_prepared/games.csv')

gamesF['tags'] = gamesF['tags'].apply(ast.literal_eval)

In [2]:
users

,user_id,products,reviews
0,6366584,1597,107
1,11402551,444,108
2,5047735,1941,113
3,5337314,1252,182
4,5382866,2723,87
...,...,...,...
7634,8340518,433,102
7635,8458227,405,214
7636,8706224,211,95
7637,8769179,449,111


In [3]:
ratings

,app_id,helpful,is_recommended,hours,user_id
0,1325200,0,1,94.5,11593837
1,1063660,0,1,14.1,3274203
2,536270,0,1,25.8,13953138
3,1544020,3,0,3.8,9650854
4,1850570,12,1,72.5,5361677
...,...,...,...,...,...
1216810,391220,0,1,26.0,7195474
1216811,253430,0,0,1.0,2444255
1216812,897450,0,1,6.0,8857084
1216813,1818450,0,1,1.0,13261742


In [4]:
gamesF

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_original,tags
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,"[Action, Adventure, Parkour, Third Person, Gre..."
1,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
2,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,"[Zombies, Adventure, Survival, Action, Third P..."
3,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."
4,250180,METAL SLUG 3,2015-09-14,True,False,False,Very Positive,90,5579,7.99,"[Arcade, Classic, Action, Co-op, Side Scroller..."
...,...,...,...,...,...,...,...,...,...,...,...
34315,1953060,Time Thief,2022-05-24,True,True,True,Very Positive,97,171,0.00,"[Action-Adventure, Action, Adventure, Explorat..."
34316,668600,Pixelum,2020-10-01,True,False,False,Mixed,41,58,0.00,"[RPG, Adventure, Simulation, Indie, Early Acce..."
34317,2071510,Goblin Rules Football,2022-10-20,True,True,True,Mostly Positive,72,11,0.00,"[Sports, Arcade, 2D, Football (American), Team..."
34318,962400,Granny,2018-11-20,True,False,False,Very Positive,90,1403,4.99,"[Horror, Survival Horror, Adventure, First-Per..."


In [5]:
from sklearn.model_selection import train_test_split


train_ratings, test_ratings = train_test_split(
    ratings, test_size=0.15, random_state=21
)

Split but make sure that all users appear in train set for predict, model learn

In [6]:
users_in_validation = train_ratings["user_id"].unique()
all_users = users["user_id"].unique()

print(f"There are {len(users_in_validation)} users in test set.")
print(f"Total number of users: {len(all_users)}")

There are 7639 users in test set.
Total number of users: 7639


In [7]:
train_ratings

,app_id,helpful,is_recommended,hours,user_id
748176,485860,5,0,13.7,5346670
532394,108800,2,0,6.5,8177238
1031797,380140,3,1,1.9,11877072
172187,107100,0,1,31.8,5090666
257063,257420,0,1,30.7,5023269
...,...,...,...,...,...
566392,239200,0,1,0.2,4353125
63856,238460,6,1,50.8,7005044
1130544,340000,0,1,1.0,7372560
664324,791330,0,1,8.6,8843670


In [8]:
user_counts = train_ratings['user_id'].value_counts()
user_counts = user_counts.sort_values(ascending=True)

print(user_counts)


user_id
7808390       58
4833262       60
10307665      60
5902154       60
5823763       60
            ... 
4457971     2217
4318160     2219
11656130    3217
5112758     3434
11764552    5067
Name: count, Length: 7639, dtype: int64


In [9]:
game_index_by_id = {id: i for i, id in enumerate(gamesF["app_id"])}

In [10]:
gamesF

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_original,tags
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,"[Action, Adventure, Parkour, Third Person, Gre..."
1,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
2,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,"[Zombies, Adventure, Survival, Action, Third P..."
3,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."
4,250180,METAL SLUG 3,2015-09-14,True,False,False,Very Positive,90,5579,7.99,"[Arcade, Classic, Action, Co-op, Side Scroller..."
...,...,...,...,...,...,...,...,...,...,...,...
34315,1953060,Time Thief,2022-05-24,True,True,True,Very Positive,97,171,0.00,"[Action-Adventure, Action, Adventure, Explorat..."
34316,668600,Pixelum,2020-10-01,True,False,False,Mixed,41,58,0.00,"[RPG, Adventure, Simulation, Indie, Early Acce..."
34317,2071510,Goblin Rules Football,2022-10-20,True,True,True,Mostly Positive,72,11,0.00,"[Sports, Arcade, 2D, Football (American), Team..."
34318,962400,Granny,2018-11-20,True,False,False,Very Positive,90,1403,4.99,"[Horror, Survival Horror, Adventure, First-Per..."


In [11]:
import numpy as np

# Extract all unique tags from the 'tags' column in gamesF
unique_tags = set(tag for tags_list in gamesF['tags'] for tag in tags_list)

# Convert the set to a sorted list
tags = sorted(unique_tags)

# Create an index mapping for the tags
tag_index_by_name = {name: i for i, name in enumerate(tags)}

# Build a binary array for game tags
game_features = np.zeros((len(gamesF), len(tags)))

for i, game_tags in enumerate(gamesF["tags"]):
    for tag in game_tags:
        if tag in tag_index_by_name:  # Ensure the tag is in the list of defined tags
            tag_index = tag_index_by_name[tag]
            game_features[i, tag_index] = 1

In [12]:
len(tags)

441

In [13]:
tag_index_by_name = {name: i for i, name in enumerate(tags)}

# Build a binary array for game tags
game_features = np.zeros((len(gamesF), len(tags)))

for i, game_tags in enumerate(gamesF["tags"]):
    for tag in game_tags:
        if tag in tag_index_by_name:  # Ensure the tag is in the list of defined tags
            tag_index = tag_index_by_name[tag]
            game_features[i, tag_index] = 1

In [14]:
len(game_index_by_id)

34320

In [15]:
game_features[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

## Model

Ridge Regression works well when all features contribute a bit to the output, while Lasso is preferred when you believe that only a subset of features are important.

In [36]:
users

,user_id,products,reviews
0,6366584,1597,107
1,11402551,444,108
2,5047735,1941,113
3,5337314,1252,182
4,5382866,2723,87
...,...,...,...
8048,8458227,405,214
8049,8507454,374,95
8050,8706224,211,95
8051,8769179,449,111


In [38]:
users[users['user_id']==6366584]

,user_id,products,reviews
0,6366584,1597,107


In [40]:
ratings[ratings['user_id']==6366584]

,app_id,helpful,is_recommended,hours,user_id
29730,282070,3,1,44.6,6366584
97432,287450,13,1,8.5,6366584
103710,200510,8,1,103.2,6366584
194108,282140,2,1,9.8,6366584
195836,110800,4,0,3.0,6366584
...,...,...,...,...,...
1225421,65930,3,1,20.0,6366584
1234334,47870,2,1,43.0,6366584
1243135,57650,4,0,40.0,6366584
1252806,39690,0,0,37.0,6366584


In [41]:
gamesF[gamesF['app_id'] == 10]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_original,tags
33286,10,Counter-Strike,2000-11-01,True,True,True,Overwhelmingly Positive,97,136118,9.99,"[Action, FPS, Multiplayer, Shooter, Classic, T..."


In [16]:
from sklearn.linear_model import Ridge

def train_user_model(user_id):
    user_ratings = train_ratings[train_ratings["user_id"] == user_id]
    game_indexes = [
        game_index_by_id[game_id] for game_id in user_ratings["app_id"]
    ]
    train_data = game_features[game_indexes]  # take tag of game has rating
    train_label = user_ratings["is_recommended"] # rating of that person
    model = Ridge(alpha=0.1)
    model.fit(train_data, train_label)
    return model

# build model for each user
user_model_dict = {}
for user_id in users["user_id"].unique():
    user_model_dict[user_id] = train_user_model(user_id)

In [17]:
def predict(game_id, user_id):
    game_feature = game_features[game_index_by_id[game_id]].reshape((1, -1))
    pred = user_model_dict[user_id].predict(game_feature)
    pred = pred.item()  
    
    return min(max(pred, 0), 1)

In [ ]:
# def predict(game_id, user_id):
#     game_feature = game_features[game_index_by_id[game_id]].reshape((1, -1))
#     pred = user_model_dict[user_id].predict(game_feature)
    
#     # Use a threshold to convert to binary class
#     binary_pred = 1 if pred >= 0.5 else 0
    
#     return binary_pred

In [18]:
from sklearn.metrics import root_mean_squared_error

def eval_rmse(ratings):
    predictions = np.zeros(len(ratings))
    for index, row in enumerate(ratings.itertuples(index=False)):
        predictions[index] = predict(row[0], row[4])
    rmse = root_mean_squared_error(ratings["is_recommended"], predictions)
    return float(rmse)
    
print(f"RMSE train: {eval_rmse(train_ratings)}")
print(f"RMSE validation: {eval_rmse(test_ratings)}")

RMSE train: 0.11584488037603018
RMSE validation: 0.4610557883767464


In [20]:
from sklearn.metrics import root_mean_squared_error

def eval_rmse(ratings):
    predictions = np.zeros(len(ratings))
    for index, row in enumerate(ratings.itertuples(index=False)):
        predictions[index] = predict(row[0], row[4])
    rmse = root_mean_squared_error(ratings["is_recommended"], predictions)
    return float(rmse)
    
print(f"RMSE train: {eval_rmse(train_ratings)}")
print(f"RMSE validation: {eval_rmse(test_ratings)}")

RMSE train: 0.12020606399317581
RMSE validation: 0.5234694455842915


In [21]:
user_id = 6366584
for tag, coef in zip(tags, user_model_dict[user_id].coef_):
    print("{:15s}: {:.3f}".format(tag, coef))

1980s          : 0.000
1990's         : 0.034
2.5D           : 0.021
2D             : 0.272
2D Fighter     : 0.000
2D Platformer  : 0.000
360 Video      : 0.000
3D             : 0.182
3D Fighter     : 0.000
3D Platformer  : 0.000
3D Vision      : -0.367
4 Player Local : 0.028
4X             : -0.211
6DOF           : 0.000
8-bit Music    : 0.000
ATV            : 0.000
Abstract       : 0.000
Action         : 0.017
Action RPG     : -0.151
Action RTS     : 0.146
Action Roguelike: 0.000
Action-Adventure: 0.265
Addictive      : -0.055
Adventure      : 0.040
Agriculture    : 0.000
Aliens         : 0.181
Alternate History: 0.140
Ambient        : 0.000
America        : -0.100
Animation & Modeling: 0.000
Anime          : 0.170
Arcade         : 0.047
Archery        : 0.000
Arena Shooter  : 0.000
Artificial Intelligence: 0.000
Assassin       : 0.039
Asymmetric VR  : 0.000
Asynchronous Multiplayer: 0.000
Atmospheric    : 0.068
Audio Production: 0.000
Auto Battler   : 0.000
Automation     : 0.000
Au

In [22]:
user_model_dict[user_id].intercept_.item() # this is b[i], bias

-0.36876589615623956

These are some high scores, showing that we should recommend games in these tags. For other users whose coefficients are randomly low but the system always avoid suggestions. This is clearly unreasonable because this is just a random value, the system never knows the user's interest in these games.

In [ ]:
ratings["app_id"].value_counts()

app_id
203160     2385
319630     2064
220        1803
50300      1733
241930     1644
           ... 
1157810       1
1605590       1
1269310       1
1332840       1
1041710       1
Name: count, Length: 34580, dtype: int64

In [ ]:
app_id_counts = ratings["app_id"].value_counts()
single_occurrence_count = (app_id_counts == 1).sum()

print("Number of app_ids that appear only once:", single_occurrence_count)

Number of app_ids that appear only once: 2671
